### Experiments with Large Language Models

In [1]:
# General imports
import pandas as pd
import numpy as np
from tqdm import tqdm

#  PyTorch imports
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

# Transformers imports
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedTokenizer
from transformers import Trainer, TrainingArguments, AdamW

# Types
from typing import List, Dict, Tuple, Union

# Set seed
torch.manual_seed(42)
np.random.seed(42)



#### Data

In [2]:
wiki = []
with open('data/wiki.txt', 'r', encoding='utf8') as f:
    for line in f:
        wiki.append(line)

wiki = pd.DataFrame(wiki, columns=['text'])
wiki.head()

,text
0,Khatchig Mouradian. Khatchig Mouradian is a jo...
1,Jacob Henry Studer. Jacob Henry Studer (26 Feb...
2,"John Stephen. Born in Glasgow, Stephen became ..."
3,Georgina Willis. Georgina Willis is an award w...
4,Stanley Corrsin. Corrsin was born on 3 April 1...


In [3]:
# Read tsv file
birth_places_train = pd.read_csv('data/birth_places_train.tsv', sep='\t') 
birth_places_train.head()

,Where was Khatchig Mouradian born?,Lebanon
0,Where was Jacob Henry Studer born?,Columbus
1,Where was John Stephen born?,Glasgow
2,Where was Georgina Willis born?,Australia
3,Where was Stanley Corrsin born?,Philadelphia
4,Where was Eduard Ender born?,Rome


In [4]:
birth_places_test = pd.read_csv('data/birth_places_test.tsv', sep='\t')
birth_places_test.head()

,Where was Bryan Dubreuiel born?,Atlanta
0,Where was Ralf Wadephul born?,Berlin
1,Where was Joseph Baggaley born?,England
2,Where was Sandhya Sanjana born?,Mumbai
3,Where was Alfred Mele born?,Detroit
4,Where was Murray Esler born?,Geelong


### Build a single file dataset with $(q, c, a)$ structure

In [5]:
def build_dataset_file(
    qa_df: pd.DataFrame, # questions and answers
    wiki_df: pd.DataFrame, # context
    filename: str # name of the file to be created
) -> None:

    """
    Build dataset file for T5 training
    """
    qa_df_values = qa_df.values
    wiki_df_values = wiki_df.values

    matches = {}
    for i in tqdm(range(len(qa_df_values)), desc='Matching questions and contexts', total=len(qa_df_values)):
        [q, a] = qa_df_values[i]
        person = q.split(' ')[2:-1] # Get the name of the person
        person = ' '.join(person) # Join the name of the person

        # Find the context of the person
        for j in range(len(wiki_df_values)):
            c = wiki_df_values[j][0] # There is only one column in the wiki DataFrame
            if person in c:
                # Remove new line characters from context
                c = c.replace('\n', '')
                matches[person] = (q, c, a)
                break
    
    print(f'Number of entries preprocessed: {len(matches.keys())}')

    # Build the dataset file
    # Create a new csv file with the columns: question, context, answer
    with open(f'data/{filename}.tsv', 'w', encoding='utf8') as f:
        for k in matches.keys():
            q, c, a = matches[k]
            # Write the question, context and answer in the file on the same line
            f.write(f'{q}\t{c}\t{a}\n')
            
    

In [6]:
build_dataset_file(birth_places_train, wiki, 'birth_places_train_clean')
build_dataset_file(birth_places_test, wiki, 'birth_places_test_clean')

Matching questions and contexts: 100%|██████████| 1999/1999 [00:00<00:00, 2966.02it/s]


Number of entries preprocessed: 1990


Matching questions and contexts: 100%|██████████| 499/499 [00:00<00:00, 1261.25it/s]

Number of entries preprocessed: 499


In [7]:
# Read the csv file
train_dataset_df = pd.read_csv('data/birth_places_train_clean.tsv', sep='\t', names=['question', 'context', 'answer'])
train_dataset_df.head()

,question,context,answer
0,Where was Jacob Henry Studer born?,Jacob Henry Studer. Jacob Henry Studer (26 Feb...,Columbus
1,Where was John Stephen born?,"John Stephen. Born in Glasgow, Stephen became ...",Glasgow
2,Where was Georgina Willis born?,Georgina Willis. Georgina Willis is an award w...,Australia
3,Where was Stanley Corrsin born?,Stanley Corrsin. Corrsin was born on 3 April 1...,Philadelphia
4,Where was Eduard Ender born?,Eduard Ender. Eduard Ender (3 March 1822 Rome ...,Rome


In [8]:
test_dataset_df = pd.read_csv('data/birth_places_test_clean.tsv', sep='\t', names=['question', 'context', 'answer'])
test_dataset_df.head()

,question,context,answer
0,Where was Ralf Wadephul born?,"Ralf Wadephul. Ralf Wadephul, born 1958 in Ber...",Berlin
1,Where was Joseph Baggaley born?,Joseph Baggaley. Joseph Baggaley (c. 1884 -- 1...,England
2,Where was Sandhya Sanjana born?,Sandhya Sanjana. Sandhya Sanjana (); is a sing...,Mumbai
3,Where was Alfred Mele born?,"Alfred Mele. Born in Detroit, Michigan, Mele a...",Detroit
4,Where was Murray Esler born?,Murray Esler. Professor Murray David Esler (bo...,Geelong


### Create utility functions for the $(q, c, a)$ scenario

In [9]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(entry: Tuple[str, str, str]) -> Tuple[str, str]:
    question, context, answer = entry
    result = {}
    result['input_text'] = 'question: %s  context: %s </s>' % (question, context)
    result['target_text'] = '%s </s>' % (answer)
    return result

# tokenize the examples
def convert_to_features(qa_entry: Tuple[str, str, str], tokenizer: PreTrainedTokenizer) -> Dict[str, List[int]]:
    input_encodings = tokenizer.encode_plus(text=qa_entry['input_text'], padding='max_length', max_length=512, truncation=True) # the context is limited to 512 tokens
    target_encodings = tokenizer.encode_plus(text=qa_entry['target_text'], padding='max_length', max_length=32, truncation=True) # the answer is limited to 32 tokens, which should conver most of the answers 

    encodings = {
        'input_ids': np.array(input_encodings['input_ids']), 
        'attention_mask': np.array(input_encodings['attention_mask']),
        'target_ids': np.array(target_encodings['input_ids']),
        'target_attention_mask': np.array(target_encodings['attention_mask'])
    }

    return encodings

In [10]:
# Test the functions on a sample
tokenizer = T5Tokenizer.from_pretrained('t5-small')
sample = train_dataset_df.sample(1)
sample = add_eos_to_examples(sample.values[0])
sample = convert_to_features(sample, tokenizer)

c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\models\t5\tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

In [11]:
class BirthPlaceDataset(Dataset):
    """
    Dataset for birth place prediction made on a (q, c, a) format,
    where q is the question, c is the context and a is the answer.

    Args:
        df (pd.DataFrame): DataFrame containing the questions, contexts and answers
    """
    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df
        self.df_values = self.df.values
        self.tokenizer = T5Tokenizer.from_pretrained('t5-small')

    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict[str, List[int]]:
        entry = self.df_values[idx]
        entry = add_eos_to_examples(entry)
        entry = convert_to_features(entry, self.tokenizer)
        return entry

In [12]:
# Create two datasets, one for training and one for validation
train_dataset = BirthPlaceDataset(train_dataset_df)
val_dataset = BirthPlaceDataset(test_dataset_df)

# Check if each element in the list of batches has the same shape

# Check for input_ids
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['input_ids'].shape != (512,):
        print(f'Entry {i} has a different shape for input_ids: {entry["input_ids"].shape}')

# Check for attention_mask
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['attention_mask'].shape != (512,):
        print(f'Entry {i} has a different shape for attention_mask: {entry["attention_mask"].shape}')

# Check for target_ids
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['target_ids'].shape != (32,):
        print(f'Entry {i} has a different shape for target_ids: {entry["target_ids"].shape}')

# Check for target_attention_mask
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['target_attention_mask'].shape != (32,):
        print(f'Entry {i} has a different shape for target_attention_mask: {entry["target_attention_mask"].shape}')

#### A sample from new the new dataset

In [13]:
print(f'Number of training examples: {len(train_dataset)}')
print(f'Number of validation examples: {len(val_dataset)}')

Number of training examples: 1990
Number of validation examples: 499


In [14]:
## Load t5 small model for question answering
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

In [15]:
tokenizer = T5Tokenizer.from_pretrained('t5-small', return_dict=True)

#### Testing the t5 model

In [16]:
# Test model
input_ids = tokenizer.encode("translate English to German: How old are you?", return_tensors="pt")
outputs = model.generate(input_ids, max_length=40, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Wie alt sind Sie?


#### [ Task 1 ] Cold evaluation of the pretrained T5-small model

The evaluation script was taken from SQuAD dataset and it validates the exact matches of the model answers compared with the real ones.

In [17]:
# Test dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Test is each batch has the correct shape
for batch in train_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    target_ids = batch['target_ids'].to(device)
    target_attention_mask = batch['target_attention_mask'].to(device)
    assert input_ids.shape == (8, 512), f'input_ids has a shape of {input_ids.shape}'
    assert attention_mask.shape == (8, 512), f'attention_mask has a shape of {attention_mask.shape}'
    assert target_ids.shape == (8, 32), f'target_ids has a shape of {target_ids.shape}'
    assert target_attention_mask.shape == (8, 32), f'target_attention_mask has a shape of {target_attention_mask.shape}'
    break

#### Evaluation script

In [18]:
from collections import Counter
import string
import re


def normalize_answer(s: str) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [19]:
answers = []
for batch in tqdm(val_dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=32,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

100%|██████████| 63/63 [03:32<00:00,  3.37s/it]


In [39]:
predictions = []
references = []
for ref, pred in zip(val_dataset, answers):
  a = ref['target_ids']  
  a = tokenizer.decode(a)

  # Remove padding
  a = a.replace('<pad>', '')
  a = a.replace('</s>', '')
  a = a.replace('<s>', '')
  
  # Remove all whitespace from the beginning and end
  a = a.rstrip()

  pred = pred.replace('<pad>', '')
  pred = pred.replace('</s>', '')
  pred = pred.replace('<s>', '')
  
  # Remove all whitespace
  pred = pred.rstrip()

  # Remove the space at the beginning of all predictions
  if pred[0] == ' ': 
    pred = pred[1:]

  predictions.append(pred)
  references.append(a)
  

In [40]:
predictions[11], references[11]

('Oxford, England', 'Washington')

In [41]:
evaluate(references, predictions)

{'exact_match': 0.0, 'f1': 0.10020040080160321}

#### Conclusion on preliminary results

TBD

#### [Task 2] Finetuning process

### Tracing the model

Here there are two different possibilities:
1. Training with the HuggingFace API (i.e Trainer)
2. Training in the classical manner or in our case the PyTorch way

For this model, in order to have better tracktability I will use the second option.

In [36]:
# Create the new training environment
finetune_model = model # use the same model as before
finetune_tokenizer = tokenizer # use the same tokenizer as before

# Create the new optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Data environment
finetune_train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
finetune_val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if the device is GPU ~ only for colab
assert device == 'cuda', 'You need to use GPU for this notebook, reset the runtime and select GPU'


c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AssertionError: You need to use GPU for this notebook, reset the runtime and select GPU

In [ ]:
# Free cuda memory
torch.cuda.empty_cache()

In [35]:
# Train the model
for epoch in range(num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    finetune_model.to(device)
    finetune_model.train()
    epoch_loss_values = []

    for batch in tqdm(finetune_train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)
        
        # Convert tensors from type Int to Long
        input_ids = input_ids.long()
        attention_mask = attention_mask.long()
        target_ids = target_ids.long()
        target_attention_mask = target_attention_mask.long()

        # Forward pass
        outputs = finetune_model(input_ids=input_ids, 
                                 attention_mask=attention_mask, 
                                 decoder_input_ids=target_ids, 
                                 decoder_attention_mask=target_attention_mask,
                                 labels=target_ids)
        
        # Calculate loss
        loss = outputs[0]
        epoch_loss_values.append(loss.item())
        
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        optimizer.zero_grad()

        del loss
    
    # Print a beutiful format of the loss
    print(f'Epoch: {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}')
    
    # Free up memory
    torch.cuda.empty_cache()

# Save the model
finetune_model.to('cpu')
finetune_model.save_pretrained('models/finetune_model_{}.pt'.format(epoch+1))


  0%|          | 0/63 [00:47<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
finetune_model.eval()
finetune_model.to(device)

answers = []
for batch in tqdm(finetune_val_dataloader):
    outs = finetune_model.generate(input_ids=batch['input_ids'], 
                            attention_mask=batch['attention_mask'],
                            max_length=32,
                            early_stopping=True)
    outs = [tokenizer.decode(ids) for ids in outs]
    answers.extend(outs)

predictions = []
references = []
for ref, pred in zip(val_dataset, answers):
    a = ref['target_ids']  
    a = tokenizer.decode(a)

    # Remove padding
    a = a.replace('<pad>', '')
    a = a.replace('</s>', '')
    a = a.replace('<s>', '')
    
    # Remove all whitespace from the beginning and end
    a = a.rstrip()

    pred = pred.replace('<pad>', '')
    pred = pred.replace('</s>', '')
    pred = pred.replace('<s>', '')
    
    # Remove all whitespace
    pred.replace(' ', '')

    predictions.append(pred)
    references.append(a)

evaluate(references, predictions)